In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import collections
import string
import numpy as np
from numpy import matrix
from numpy import linalg
%matplotlib inline

from multiprocessing import Pool


from cipher import *
from cipherbreak import *

c7b = open('2016/7b.ciphertext').read()

In [108]:
c7bs = sanitise(c7b)
c7br = cat(reversed(c7bs))

In [2]:
def bifid_grid(keyword, wrap_alphabet, letter_mapping):
    cipher_alphabet = keyword_cipher_alphabet_of(keyword, wrap_alphabet)
    if letter_mapping is None:
        letter_mapping = {'j': 'i'}
    translation = ''.maketrans(letter_mapping)
    cipher_alphabet = cat(collections.OrderedDict.fromkeys(cipher_alphabet.translate(translation)))
    f_grid = {k: ((i // 5) + 1, (i % 5) + 1) 
              for i, k in enumerate(cipher_alphabet)}
    r_grid = {((i // 5) + 1, (i % 5) + 1): k 
              for i, k in enumerate(cipher_alphabet)}
    return translation, f_grid, r_grid

In [156]:
import pprint

In [157]:
pprint.pprint(bifid_grid('iguana', KeywordWrapAlphabet.from_a, None))

({106: 'i'},
 {'a': (1, 4),
  'b': (2, 1),
  'c': (2, 2),
  'd': (2, 3),
  'e': (2, 4),
  'f': (2, 5),
  'g': (1, 2),
  'h': (3, 1),
  'i': (1, 1),
  'k': (3, 2),
  'l': (3, 3),
  'm': (3, 4),
  'n': (1, 5),
  'o': (3, 5),
  'p': (4, 1),
  'q': (4, 2),
  'r': (4, 3),
  's': (4, 4),
  't': (4, 5),
  'u': (1, 3),
  'v': (5, 1),
  'w': (5, 2),
  'x': (5, 3),
  'y': (5, 4),
  'z': (5, 5)},
 {(1, 1): 'i',
  (1, 2): 'g',
  (1, 3): 'u',
  (1, 4): 'a',
  (1, 5): 'n',
  (2, 1): 'b',
  (2, 2): 'c',
  (2, 3): 'd',
  (2, 4): 'e',
  (2, 5): 'f',
  (3, 1): 'h',
  (3, 2): 'k',
  (3, 3): 'l',
  (3, 4): 'm',
  (3, 5): 'o',
  (4, 1): 'p',
  (4, 2): 'q',
  (4, 3): 'r',
  (4, 4): 's',
  (4, 5): 't',
  (5, 1): 'v',
  (5, 2): 'w',
  (5, 3): 'x',
  (5, 4): 'y',
  (5, 5): 'z'})


In [158]:
pprint.pprint(bifid_grid('z', KeywordWrapAlphabet.from_a, None))

({106: 'i'},
 {'a': (1, 2),
  'b': (1, 3),
  'c': (1, 4),
  'd': (1, 5),
  'e': (2, 1),
  'f': (2, 2),
  'g': (2, 3),
  'h': (2, 4),
  'i': (2, 5),
  'k': (3, 1),
  'l': (3, 2),
  'm': (3, 3),
  'n': (3, 4),
  'o': (3, 5),
  'p': (4, 1),
  'q': (4, 2),
  'r': (4, 3),
  's': (4, 4),
  't': (4, 5),
  'u': (5, 1),
  'v': (5, 2),
  'w': (5, 3),
  'x': (5, 4),
  'y': (5, 5),
  'z': (1, 1)},
 {(1, 1): 'z',
  (1, 2): 'a',
  (1, 3): 'b',
  (1, 4): 'c',
  (1, 5): 'd',
  (2, 1): 'e',
  (2, 2): 'f',
  (2, 3): 'g',
  (2, 4): 'h',
  (2, 5): 'i',
  (3, 1): 'k',
  (3, 2): 'l',
  (3, 3): 'm',
  (3, 4): 'n',
  (3, 5): 'o',
  (4, 1): 'p',
  (4, 2): 'q',
  (4, 3): 'r',
  (4, 4): 's',
  (4, 5): 't',
  (5, 1): 'u',
  (5, 2): 'v',
  (5, 3): 'w',
  (5, 4): 'x',
  (5, 5): 'y'})


In [159]:
pprint.pprint(bifid_grid('iguana', KeywordWrapAlphabet.from_a, {'q': 'p'}))

({113: 'p'},
 {'a': (1, 4),
  'b': (2, 1),
  'c': (2, 2),
  'd': (2, 3),
  'e': (2, 4),
  'f': (2, 5),
  'g': (1, 2),
  'h': (3, 1),
  'i': (1, 1),
  'j': (3, 2),
  'k': (3, 3),
  'l': (3, 4),
  'm': (3, 5),
  'n': (1, 5),
  'o': (4, 1),
  'p': (4, 2),
  'r': (4, 3),
  's': (4, 4),
  't': (4, 5),
  'u': (1, 3),
  'v': (5, 1),
  'w': (5, 2),
  'x': (5, 3),
  'y': (5, 4),
  'z': (5, 5)},
 {(1, 1): 'i',
  (1, 2): 'g',
  (1, 3): 'u',
  (1, 4): 'a',
  (1, 5): 'n',
  (2, 1): 'b',
  (2, 2): 'c',
  (2, 3): 'd',
  (2, 4): 'e',
  (2, 5): 'f',
  (3, 1): 'h',
  (3, 2): 'j',
  (3, 3): 'k',
  (3, 4): 'l',
  (3, 5): 'm',
  (4, 1): 'o',
  (4, 2): 'p',
  (4, 3): 'r',
  (4, 4): 's',
  (4, 5): 't',
  (5, 1): 'v',
  (5, 2): 'w',
  (5, 3): 'x',
  (5, 4): 'y',
  (5, 5): 'z'})


In [87]:
# def bifid_decipher(message, keyword, wrap_alphabet=KeywordWrapAlphabet.from_a, 
#                    letter_mapping=None, period=None):
#     translation, f_grid, r_grid = bifid_grid(keyword, wrap_alphabet, letter_mapping)
    
#     t_message = message.translate(translation)
#     pairs0 = [f_grid[l] for l in t_message]
#     items = sum([list(p) for p in pairs0], [])
#     gap = len(message)
#     pairs1 = [(items[i//2], items[i//2+gap]) for i in range(0, len(items), 2)]
#     return cat(r_grid[p] for p in pairs1)
    

In [162]:
def bifid_encipher(message, keyword, wrap_alphabet=KeywordWrapAlphabet.from_a, 
                   letter_mapping=None, period=None, fillvalue=None):
    translation, f_grid, r_grid = bifid_grid(keyword, wrap_alphabet, letter_mapping)
    
    t_message = message.translate(translation)
    pairs0 = [f_grid[l] for l in sanitise(t_message)]
    if period:
        chunked_pairs = [pairs0[i:i+period] for i in range(0, len(pairs0), period)]
        if len(chunked_pairs[-1]) < period and fillvalue:
            chunked_pairs[-1] += [f_grid[fillvalue]] * (period - len(chunked_pairs[-1]))
    else:
        chunked_pairs = [pairs0]
    
    pairs1 = []
    for c in chunked_pairs:
        items = sum(list(list(i) for i in zip(*c)), [])
        p = [(items[i], items[i+1]) for i in range(0, len(items), 2)]
        pairs1 += p
    
    return cat(r_grid[p] for p in pairs1)
    

In [163]:
bifid_encipher('this is a test message for the keyword decipherment', 'elephant', wrap_alphabet=KeywordWrapAlphabet.from_last)

'nkklawamdkoedysipdesltirsnoesqlvvaloderbhel'

In [83]:
ot, ofg, org = bifid_grid('iguana', KeywordWrapAlphabet.from_a, None)

In [85]:
op0 = [ofg[l] for l in "indiacurry"]
op0

[(1, 1),
 (1, 5),
 (2, 3),
 (1, 1),
 (1, 4),
 (2, 2),
 (1, 3),
 (4, 3),
 (4, 3),
 (5, 4)]

In [86]:
ocp = chunks(op0, 4, fillvalue=[[ofg['a']]])
acc = []
ocp

[[(1, 1), (1, 5), (2, 3), (1, 1)],
 [(1, 4), (2, 2), (1, 3), (4, 3)],
 [(4, 3), (5, 4), (1, 4), (1, 4)]]

In [87]:
acc=[]
for c in ocp:
    items = sum(list(list(i) for i in zip(*c)), [])
    p = [(items[i], items[i+1]) for i in range(0, len(items), 2)]
    acc += p
acc

[(1, 1),
 (2, 1),
 (1, 5),
 (3, 1),
 (1, 2),
 (1, 4),
 (4, 2),
 (3, 3),
 (4, 5),
 (1, 1),
 (3, 4),
 (4, 4)]

In [88]:
cat(org[p] for p in acc)

'ibnhgaqltims'

In [164]:
def bifid_decipher(message, keyword, wrap_alphabet=KeywordWrapAlphabet.from_a, 
                   letter_mapping=None, period=None, fillvalue=None):
    translation, f_grid, r_grid = bifid_grid(keyword, wrap_alphabet, letter_mapping)
    
    t_message = message.translate(translation)
    pairs0 = [f_grid[l] for l in sanitise(t_message)]
    if period:
        chunked_pairs = [pairs0[i:i+period] for i in range(0, len(pairs0), period)]
        if len(chunked_pairs[-1]) < period and fillvalue:
            chunked_pairs[-1] += [f_grid[fillvalue]] * (period - len(chunked_pairs[-1]))
    else:
        chunked_pairs = [pairs0]
        
    pairs1 = []
    for c in chunked_pairs:
        items = [j for i in c for j in i]
        gap = len(c)
        p = [(items[i], items[i+gap]) for i in range(gap)]
        pairs1 += p

    return cat(r_grid[p] for p in pairs1)    

In [5]:
bifid_grid('iguana', KeywordWrapAlphabet.from_a, None)

({106: 'i'},
 {'a': (1, 4),
  'b': (2, 1),
  'c': (2, 2),
  'd': (2, 3),
  'e': (2, 4),
  'f': (2, 5),
  'g': (1, 2),
  'h': (3, 1),
  'i': (1, 1),
  'k': (3, 2),
  'l': (3, 3),
  'm': (3, 4),
  'n': (1, 5),
  'o': (3, 5),
  'p': (4, 1),
  'q': (4, 2),
  'r': (4, 3),
  's': (4, 4),
  't': (4, 5),
  'u': (1, 3),
  'v': (5, 1),
  'w': (5, 2),
  'x': (5, 3),
  'y': (5, 4),
  'z': (5, 5)},
 {(1, 1): 'i',
  (1, 2): 'g',
  (1, 3): 'u',
  (1, 4): 'a',
  (1, 5): 'n',
  (2, 1): 'b',
  (2, 2): 'c',
  (2, 3): 'd',
  (2, 4): 'e',
  (2, 5): 'f',
  (3, 1): 'h',
  (3, 2): 'k',
  (3, 3): 'l',
  (3, 4): 'm',
  (3, 5): 'o',
  (4, 1): 'p',
  (4, 2): 'q',
  (4, 3): 'r',
  (4, 4): 's',
  (4, 5): 't',
  (5, 1): 'v',
  (5, 2): 'w',
  (5, 3): 'x',
  (5, 4): 'y',
  (5, 5): 'z'})

In [139]:
bifid_encipher("indiajelly", 'iguana')

'ibidonhprm'

In [140]:
bifid_decipher('ibidonhprm', 'iguana')

'indiaielly'

In [137]:
bifid_encipher("indiacurry", 'iguana', period=4)

'ibnhgaqltm'

In [138]:
bifid_decipher("ibnhgaqltm", 'iguana', period=4)

'indiacurry'

In [144]:
bifid_encipher("indiacurry", 'iguana', period=4, fillvalue='x')

'ibnhgaqltzml'

In [146]:
bifid_decipher("ibnhgaqltzml", 'iguana', period=4)

'indiacurryxx'

In [101]:
p0 = [(1, 1), (2, 1), (1, 5), (3, 1)]

In [103]:
t0 = [j for i in p0 for j in i]
t0

[1, 1, 2, 1, 1, 5, 3, 1]

In [104]:
[(t0[i], t0[i+4]) for i in range(4)]

[(1, 1), (1, 5), (2, 3), (1, 1)]

In [130]:
bifid_decipher(c7bs, 'ligo', KeywordWrapAlphabet.from_a, period=4, fillvalue=None)

'martinwehavemadeadreadfulmistakeandihavebeentooslowtoadmitthattomyselfihavehadavisitfromthewomanfromthesyndicateandiconfrontedheraboutthesourceofthetemplatessheconfirmedmyworstfearsandnowiwanttocrawlawayanddiewhathavewedoneoursoftwarehasledtosomuchsufferingwhenitwasdesignedtodotheoppositeiaskedherhowthecabinetofficecouldpossiblyhaveauthorisedthisandshelaughedandexplainedthatthesyndicatenolongerworkedforthebritishgovernmentcallitprivateenterpriseshesaidwehavealwaysbeengoodatthatmyhorrormusthavebeenwrittenallovermyfaceshedidntseemsurprisedatmyreactionbutequallyshedidnttakeitwellandcivilitywasabandonediaskedherhowitcouldbelegalletalonemoraltodowhattheyproposedandheranswerwasthatitwasnecessaryisaidwewouldnthelpthemandshesaiditwasnecessarythatwedidisaidiwouldntbeabletofacemyfamilyandfriendsifwecooperatedandshesaidiwouldnthavetoworryaboutthatforlongonewayoranotherthepdssyndicateweregoingtomakesurewebothdisappearedlookingbackicanseethatfromthestartthiswholethinghasactedtodrawusintoitscentrea

In [147]:
wcat(segment(bifid_decipher(c7bs, 'ligo', KeywordWrapAlphabet.from_a, period=4)))

'martin we have made a dreadful mistake and i have been too slow to admit that to myself i have had a visit from the woman from the syndicate and i confronted her about the source of the templates she confirmed my worst fears and now i want to crawl away and die what have we done our software has led to so much suffering when it was designed to do the opposite i asked her how the cabinet office could possibly have authorised this and she laughed and explained that the syndicate no longer worked for the british government call it private enterprises he said we have always been good at that my horror must have been written all over myfaces he didnt seem surprised at my reaction but equally she didnt take it well and civility was abandoned i asked her how it could be legal let alone moral to do what they proposed and her answer was that it was necessary i said we wouldnt help them and she said it was necessary that we did i said i wouldnt be able to face my family and friends if we cooper

In [49]:
p0 = [(2, 1), (3, 3), (3, 3), (5, 1), (1, 4)]

In [54]:
items = sum([list(p) for p in p0], [])
items

[2, 1, 3, 3, 3, 3, 5, 1, 1, 4]

In [55]:
gap=5
[(items[i//2], items[i//2+gap]) for i in range(0, len(items), 2)]

[(2, 3), (1, 5), (3, 1), (3, 1), (3, 4)]

In [91]:
c7bs = sanitise(c7b)

In [115]:
def bifid_break_mp(message, wordlist=keywords, fitness=Pletters,
                     number_of_solutions=1, chunksize=500):
    """Breaks a keyword substitution cipher using a dictionary and
    frequency analysis

    >>> keyword_break_mp(keyword_encipher('this is a test message for the ' \
          'keyword decipherment', 'elephant', KeywordWrapAlphabet.from_last), \
          wordlist=['cat', 'elephant', 'kangaroo']) # doctest: +ELLIPSIS
    (('elephant', <KeywordWrapAlphabet.from_last: 2>), -52.834575011...)
    >>> keyword_break_mp(keyword_encipher('this is a test message for the ' \
          'keyword decipherment', 'elephant', KeywordWrapAlphabet.from_last), \
          wordlist=['cat', 'elephant', 'kangaroo'], \
          number_of_solutions=2) # doctest: +ELLIPSIS, +NORMALIZE_WHITESPACE
    [(('elephant', <KeywordWrapAlphabet.from_last: 2>), -52.834575011...), 
    (('elephant', <KeywordWrapAlphabet.from_largest: 3>), -52.834575011...)]
    """
    with Pool() as pool:
        helper_args = [(message, word, wrap, fitness)
                       for word in wordlist
                       for wrap in KeywordWrapAlphabet]
        # Gotcha: the helper function here needs to be defined at the top level
        #   (limitation of Pool.starmap)
        breaks = pool.starmap(bifid_break_worker, helper_args, chunksize)
        if number_of_solutions == 1:
            return max(breaks, key=lambda k: k[1])
        else:
            return sorted(breaks, key=lambda k: k[1], reverse=True)[:number_of_solutions]


In [116]:
def bifid_break_worker(message, keyword, wrap_alphabet, fitness):
    plaintext = bifid_decipher(message, keyword, wrap_alphabet)
    fit = fitness(plaintext)
    logger.debug('Keyword break attempt using key {0} (wrap={1}) gives fit of '
                 '{2} and decrypt starting: {3}'.format(keyword, 
                     wrap_alphabet, fit, sanitise(plaintext)[:50]))
    return (keyword, wrap_alphabet), fit

In [107]:
c7bs = sanitise(c7b)
c7br = cat(reversed(c7bs))
c7br

'enamokkneogiyegrkcuzbgsydkoqoswiwvtgbolrkfuzbgsyskdqusgnttqetuonyegyfkbsteqeycgbudbvqadcepgqrsbbeaeoqilrqcsosfcdyrbztiuirvqrtcmesbkudboeytksofknyegrambmctvxeogttfggemokopiqwinutqhdtoftsgsathnemlteprmrqbstdolsaklrueucsvncyhgqqcsfqlutsdxzthnfuvotauhnyegosloeogwrrqclelbonknefqkrkofvoqhxttvrimhyttvyqosrlegqtbeyroeuegbtnqkeecgqepblclvutouaehtoekceqgpeobwaohndxlstmvnvalttupquvoieruortugrhfsyqosnsqcesrvhtcrarvqnshkudbistnbwuootauhtqefiolctxqoqhdsmrneettrtberyybtnqblqxgrtrveqyfoboxwaehwlplqrcbiowhirimheyotqcciorhitvdaqkdurimhtheslelwxuooneagedooqlfibrlfhesrpxtgrrbkarhrxqobtogwaehwrimheyotlviqpesoyfkbsteqoviiynstnvhltfgarqostqdfotktpfhlrnadyegrtencsrtltknqffyqsnietwrixhogeqgsooihkiuhtwtonotknenquarhtnerycgicadbschhthlbeurbzpmqkqbsoebweprtepkhtsqhrslbrrefycoisnkumuedqrseqtshftqvftvkntlgbyqbyqosyotbucvtetvqoawhtgniucoadpiqomkeysgcyroeuegbefwimtedcpqcyogakzhbnepcztbqoviiotcnguiocdaipkqbkebwyqosepuqsrecnfsvedoipoqkmyegsqdsaogktgniurofkwfldeoupsqerncsfevoyqosxcoueswnfoqkspkbnyrheuqxotktarhasoiofnqh

In [148]:
bifid_break_mp(c7bs, wordlist=['gravity', 'ligo'])

(('ligo', <KeywordWrapAlphabet.from_a: 1>), -2505.924490942904)

In [102]:
bifid_decipher(c7bs, 'capris', KeywordWrapAlphabet.from_a )

'ksotagstmczesqstldwfasoehepicltaryruvgstiwbtylwzlanehrdmthhlqzeohpsdytllgdxfcetbiislqetoukobnterkoyonsetkodhtymxuefpdtnhnsulwnnurhiotctcrwhprbssrdblxanlrxadxgxatetsegdoeuhawberbaswolpqkrkpfcxufohcyefaeabqtkbrykbdonghsbaodvongcfdvngmeslhetnytkocenotirklsatenkdeeyoentbryuoleqoefcuxpqbsirotbogkvtqbrqgyoqamkninrfottolgmynbsekmeouwueklcpqrekyylvronsntcotrrctdvoyfthkgalscldypooicxrtpdyttohfoqtprrbgtwepsycwpswuylkedbiglsylbctcfecisumvrbyparteagabdqouuttohvbtcdtxczusxrtleburbtkapmsfctmokootcoibkclbetoaralxzdnlpanenadkhhgtsldnyrupnqsravhtpohmplgtaacuhfpttdebroaqhedgvooyyneoebrfudfodroyklhsheqheeqhdinusytmhdqqedsbbdzcylgukonttacivvvcrprteautwrxhdmczntnixhzbeasmboscsyeqdtsxxeiodohrnofaidlbabrobumelkaeuvnlylglqnfpeqklhwlqeselhameievlbeawrlnllyetoeolencrduoghqqoeqyhlqidrrvndrrwnfhmottqllpayunortyoeariuhharstrhnsfaaoqlipqkohcrnldicnlshnysnrtdbiggeonatcseqosygonehtdutacbzdbstdcrzttntrbtunrotcuewfslftkgdruetlrxrgbmvegfsqieybgqtfxaxrvqbybcaibhoeoolnpxnrdatsspxotloerkwotcheutoerrufnprncktohsqarexdccnbt

In [104]:
bifid_decipher(c7br, 'trinket', KeywordWrapAlphabet.from_a)

'dnrobchyoikvkwtucnsiganmyoanfatckweqpwxwehfdpoxqdvtworugsnpsxanmycvirrgedmuihpienwfnpwyzdffaqsxsoutvbvqlovoplenqrnktqlernruwptiswodnhaxomrtwbmpunnbnolcrvsasmcblsvawegytshecczeperucfeqlrcosbfdhdkcaeipnsligmsnapleqpoutoqgtcobyzorcehvvkkrhoxngkavnolkcrncshwetpocgnysxewemonekysintysuqrlntykpvsckgenbecfybfmycsrtzrxfckstgbirtnmnhdhcetfvuttwptpbigplyctfisoxttslhhbvtnotminnusfcqobdtnlhnpbnhxhpirckoatchwiaofaloprrbyonlouscekwlsnboyciswcoosyirodpwsgghcqhofeuosmtqfhqsztrsscwxfzwrnhhnphtpvwnurikaeoxgftoyodrmaqnosloukciskmbycicidsgeofsbrfodhsrtskwtpbtgnsmnyncrvsrhtwrsiolhlylrnntvrkenffvgfypknntxryqqgonspndmocrhcpfrnbgerwqncuiciqverfealtrnoomnwdnenepldpgatsbbffsaogoynownnotoxbeckllrnonbknycsensfleqbctonegzaccsifmnuezvlfqsurrgwviiuwrdcgoininoyseugowzkpntfctreowotnetkswlctahmgrpoypohsnoqnqooogboctspteznllttcurbcbiewnslipsshliwsybvalnlosrrtpcfzepwsnonktceovhhrnqfeilrkmsrlseoinyfsfunulffevuofwrenqbrettkrtkbnkelciibvsysafkrisombfmsqgwlfbnekikdyreipgzshnonclsrzgyceeanhlvcluucfsomraasehigprxbmfiftdnhgxhfb